In [2]:
# ! pip install flickr_api

In [18]:
from itertools import chain

In [116]:
# ! pip install scikit-learn

In [117]:
from sklearn.feature_extraction import stop_words

In [5]:
import flickr_api
flickr_api.set_keys(api_key = '##KEY##', api_secret = '##SECRET##')

In [6]:
# photos = flickr_api.Photo.search(lon=24.516951, lat=60.298078, radius='10.0', radius_units="km")
# photos = flickr_api.Photo.search(tags="nuuksio", has_geo=True)

In [92]:
def search_name_and_gps(lat, lon, name):
    gps_walker = flickr_api.Photo.search(lat=lat, lon=lon, 
                                  radius="0.2", radius_units="km", per_page=10)
    name_walker = flickr_api.Photo.search(text=name, per_page=10)
    
    return chain(gps_walker, name_walker)

In [88]:
# w = flickr_api.Walker(flickr_api.Photo.search, lat=60.2604953, lon=24.5061496,
#                       has_geo=True, radius="1", radius_units="km", per_page=500)

In [89]:
w = list(search_name_and_gps(lat=60.2604953, lon=24.5061496, name="Lepolampi"))

In [95]:
def get_picture_url(picture):
    try:
        return picture.getSizes()['Medium']['source']
    except:
        return "Nothing"

In [90]:
for item in w:
    print([t.text for t in item.getInfo()['tags']])
    print(get_picture_url(item))
    print("---")

[]
https://live.staticflickr.com/5701/30856795036_c3f07e6e0a.jpg
---
['lepolampi', 'espoo', 'finland']
https://live.staticflickr.com/231/535975763_9642add3c5.jpg
---
['lepolampi', 'espoo', 'finland']
https://live.staticflickr.com/1285/535137438_5296a05952.jpg
---
['lepolampi', 'espoo', 'finland']
https://live.staticflickr.com/1431/535133782_8399227762.jpg
---
['lepolampi', 'espoo', 'finland']
https://live.staticflickr.com/1145/535251389_000b701fd4.jpg
---
['espoo', 'lepolampi', 'finland']
https://live.staticflickr.com/1157/535240879_b0ffcea085.jpg
---
['espoo', 'lepolampi', 'finland']
https://live.staticflickr.com/1323/535240837_b86e92c1a3.jpg
---
['espoo', 'lepolampi', 'finland']
https://live.staticflickr.com/1263/535240843_4a042b6d35.jpg
---
['espoo', 'finland', 'lepolampi']
https://live.staticflickr.com/1020/535240195_cda88e9b82.jpg
---


In [85]:
def extract_tags_and_picture(item):
    return {"tags": [t.text for t in item.getInfo()['tags']], "picture_url": get_picture_url(item)}

In [77]:
import json

In [81]:
with open("nuuksio_locations.json", "r") as read_file:
    current_data = json.load(read_file)

In [83]:
def get_latlon_and_name(item):
    
    info = {}
    
    info["name"] = item["name"]
    info["lat"] = item["location"]["lat"]
    info["lon"] = item['location']['lon']
    
    return info

In [96]:
flickr_data = []
for ind, iteem in enumerate(current_data):
    info = get_latlon_and_name(iteem)
    flickr_response = list(search_name_and_gps(lat=info["lat"], lon=info["lon"], name=info["name"]))
#     tnp = extract_tags_and_picture()
    tags = [extract_tags_and_picture(pic) for pic in flickr_response]
    flickr_data.append(tags)

In [120]:
def compile_tags_and_urls(flickr_item):
    all_tags = list(set([item for sublist in [thing["tags"] for thing in flickr_item] for item in sublist if len(item) < 20 and ":" not in item and item not in stop_words.ENGLISH_STOP_WORDS]))
    pictures = [fl["picture_url"] for fl in flickr_item]
    return {"tags": all_tags, "pictures": pictures}

In [121]:
compile_tags_and_urls(flickr_data[0])

{'tags': ['uusimaa',
  'lake',
  'espoontori',
  'huuhkaja',
  'abandoned',
  'helsingin',
  '2018',
  'texture',
  'espoonkeskus',
  'jää',
  'ice',
  'building',
  'landscape',
  'finland',
  'day',
  'pitkäjärvi',
  'orange',
  'samsung',
  'room',
  'light',
  'suomi',
  'old',
  'entresse',
  'doorway',
  'oven',
  'house',
  'espoo',
  'bubobubo',
  'red',
  'talvi',
  'hvk',
  'hugovk',
  'winter',
  'december',
  'autumn',
  'window',
  'espoodecember2018',
  'järvi',
  'apartment'],
 'pictures': ['https://live.staticflickr.com/65535/40702342013_ed0012cb18.jpg',
  'https://live.staticflickr.com/65535/32672953727_c8f7679df6.jpg',
  'https://live.staticflickr.com/7899/32197615577_de5aa27db2.jpg',
  'https://live.staticflickr.com/7925/32190568567_b287e57300.jpg',
  'https://live.staticflickr.com/7911/45746385564_4d1ed0c15a.jpg',
  'https://live.staticflickr.com/7835/46469855751_a9f62d60b0.jpg',
  'https://live.staticflickr.com/4910/46279312862_b9874dc1c4.jpg',
  'https://live.stat

In [123]:
for stuff, iteeem in zip(flickr_data, current_data):
    compiled = compile_tags_and_urls(stuff)
    iteeem["flickr_tags"] = compiled["tags"]
    iteeem["flickr_picture_urls"] = compiled["pictures"]

In [125]:
current_data[1]

{'id': 'ChIJOSsORHnujUYR_XPCpc-uGU4',
 'place_details': {'opening_times': {'open_now': False,
   'periods': [{'close': {'day': 0, 'time': '1200'},
     'open': {'day': 0, 'time': '0800'}},
    {'close': {'day': 1, 'time': '2200'}, 'open': {'day': 1, 'time': '0800'}},
    {'close': {'day': 2, 'time': '2200'}, 'open': {'day': 2, 'time': '0800'}},
    {'close': {'day': 3, 'time': '2200'}, 'open': {'day': 3, 'time': '0800'}},
    {'close': {'day': 4, 'time': '2200'}, 'open': {'day': 4, 'time': '0800'}},
    {'close': {'day': 5, 'time': '2200'}, 'open': {'day': 5, 'time': '0800'}},
    {'close': {'day': 6, 'time': '2200'}, 'open': {'day': 6, 'time': '0800'}}],
   'weekday_text': ['Monday: 8:00 AM – 10:00 PM',
    'Tuesday: 8:00 AM – 10:00 PM',
    'Wednesday: 8:00 AM – 10:00 PM',
    'Thursday: 8:00 AM – 10:00 PM',
    'Friday: 8:00 AM – 10:00 PM',
    'Saturday: 8:00 AM – 10:00 PM',
    'Sunday: 8:00 AM – 12:00 PM']},
  'photo_refs': ['CmRaAAAA6H-gNEVxaO2OfXK_8Qgfhrb-LaCcwCkHgVNz37CBn5aO1o

In [126]:
with open('nuuksio_locations_w_flickr.json', 'w') as f:
    json.dump(current_data, f)